In [80]:
!pip install scikit-learn

In [81]:
!pip install nltk

In [82]:
!pip install contractions

In [83]:
!pip install email-validator

In [84]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [85]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [86]:
import pandas as pd
import nltk
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import re
from nltk.tokenize import word_tokenize
import contractions
import inflect

newsgroup_data = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)
documents = newsgroup_data.data
categories = newsgroup_data.target

df = pd.DataFrame({'text': documents, 'category': categories})

In [87]:
def rmv_emails_websites(string):
    new_str = re.sub(r"\S+@\S+", '', string)
    new_str = re.sub(r"\S+.co\S+", '', new_str)
    new_str = re.sub(r"\S+.ed\S+", '', new_str)
    new_str = re.sub(r"[0-9]+", '', new_str)
    return new_str

def sentence_simplifier(sentence):
  stemmer = PorterStemmer()
  simplified_words = [stemmer.stem(word) for word in sentence.split()]
  return " ".join(simplified_words)

def preprocess_text(input_text):
    # remove emails
    clean_text = rmv_emails_websites(input_text)
    # simplify sentences writing -> write
    clean_text = sentence_simplifier(clean_text)
    # all within square brackets
    clean_text = re.sub(r'\[.*?\]', '', clean_text)
    # all within round brackets
    clean_text = re.sub(r'\(.*?\)', '', clean_text)
    # URLs and Links
    clean_text = re.sub(r'http\S+', '', clean_text)
    # Lowercase
    clean_text = clean_text.lower()
    # white space
    clean_text = re.sub('\s+', ' ', clean_text)
    # Expand contractions: can't -> cannot
    clean_text = contractions.fix(clean_text)
    # Remove special characters
    clean_text = re.sub('[^a-zA-Z0-9\s]', '', clean_text)
    # Numbers to text
    temp = inflect.engine()
    words = []
    for word in clean_text.split():
        if word.isdigit():
            words.append(temp.number_to_words(word))
        else:
            words.append(word)
    clean_text = ' '.join(words)

    tokens = word_tokenize(clean_text)
    tokens = [token for token in tokens if token not in stop_words]
    clean_text = ' '.join(tokens)
    # Punctuation
    clean_text = re.sub(r'[^\w\s]', '', clean_text)

    return clean_text


In [88]:
dummy_text = df['text'][0]
print(dummy_text)
print('-'*30)
dummy_text = preprocess_text(dummy_text)
print(dummy_text)


From: v064mb9k@ubvmsd.cc.buffalo.edu (NEIL B. GANDLER)
Subject: Need info on 88-89 Bonneville
Organization: University at Buffalo
Lines: 10
News-Software: VAX/VMS VNEWS 1.41
Nntp-Posting-Host: ubvmsd.cc.buffalo.edu


 I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.

			Neil Gandler

------------------------------
subject need info bonnevil organization univers buffalo lines newssoftware vaxvm vnew nntppostinghost model bonnevilles heard le se lse sse ssei could someon tell differ far featur performance also curiou know book valu prefer model much less book valu usual get word much 

Takes approximately 1 minute to preprocess all the text data

In [89]:
df['clean_text'] = df['text'].apply(preprocess_text)

In [90]:
len(df)

7532

In [91]:
df.to_csv('test.csv', index=False)